# Día 2: Programación Funcional en Python

## Descripción General

La programación funcional es un paradigma que trata la computación como la evaluación de funciones matemáticas, evitando el cambio de estado y los datos mutables. Python, aunque es principalmente un lenguaje multiparadigma, proporciona herramientas poderosas para la programación funcional como `map()`, `filter()`, `reduce()`, y el módulo `functools`.

En este notebook, exploraremos las funciones de orden superior más importantes de Python y aprenderemos a usar `functools` para crear código más eficiente y expresivo. Estas técnicas son especialmente útiles en procesamiento de datos, pipelines de transformación, y optimización de rendimiento.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Utilizar `map()`, `filter()` y `reduce()` para transformar y procesar colecciones de datos
2. Aplicar `functools.partial()` para crear funciones especializadas
3. Optimizar funciones con `functools.lru_cache()` para mejorar el rendimiento
4. Combinar técnicas funcionales para crear pipelines de procesamiento de datos
5. Decidir cuándo usar programación funcional vs comprehensions o loops tradicionales

## 1. map() - Transformar Colecciones

### El Problema que Resuelve

Frecuentemente necesitamos aplicar la misma transformación a cada elemento de una colección. Sin `map()`, tendríamos que usar loops explícitos o list comprehensions, lo que puede ser menos expresivo para transformaciones simples.

`map()` aplica una función a cada elemento de un iterable y devuelve un iterador con los resultados.

### ❌ MAL: Loop Explícito

In [ ]:
# Transform list of numbers to their squares
numbers = [1, 2, 3, 4, 5]
squares = []
for num in numbers:
    squares.append(num ** 2)
print(squares)

### ✅ BIEN: Usando map()

In [ ]:
# Using map with a lambda function
numbers = [1, 2, 3, 4, 5]
squares = list(map(lambda x: x ** 2, numbers))
print(squares)

# Using map with a named function
def square(x: int) -> int:
    """Calculate the square of a number."""
    return x ** 2

squares = list(map(square, numbers))
print(squares)

# Map with multiple iterables
numbers1 = [1, 2, 3]
numbers2 = [10, 20, 30]
sums = list(map(lambda x, y: x + y, numbers1, numbers2))
print(f"Sums: {sums}")  # [11, 22, 33]

### Ejemplo Práctico: Procesamiento de Datos

In [ ]:
# Convert temperatures from Celsius to Fahrenheit
celsius_temps = [0, 10, 20, 30, 40]

def celsius_to_fahrenheit(celsius: float) -> float:
    """
    Convert Celsius to Fahrenheit.
    
    :param celsius: Temperature in Celsius
    :type celsius: float
    :return: Temperature in Fahrenheit
    :rtype: float
    """
    return (celsius * 9/5) + 32

fahrenheit_temps = list(map(celsius_to_fahrenheit, celsius_temps))
print(f"Celsius: {celsius_temps}")
print(f"Fahrenheit: {fahrenheit_temps}")

# Parse string data
string_numbers = ["1", "2", "3", "4", "5"]
int_numbers = list(map(int, string_numbers))
print(f"Parsed integers: {int_numbers}")

# Extract attributes from objects
class Person:
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

people = [Person("Alice", 30), Person("Bob", 25), Person("Charlie", 35)]
names = list(map(lambda p: p.name, people))
print(f"Names: {names}")

### Aprendizaje Clave

`map(function, iterable)` aplica una función a cada elemento de un iterable y devuelve un iterador. Es más expresivo que loops para transformaciones simples, pero las list comprehensions suelen ser más Pythonic para casos simples.

**Referencia oficial:** [Python map() built-in function](https://docs.python.org/3/library/functions.html#map)

## 2. filter() - Filtrar Colecciones

### El Problema que Resuelve

A menudo necesitamos seleccionar solo los elementos de una colección que cumplan cierta condición. `filter()` proporciona una forma funcional de hacer esto, aplicando una función predicado (que devuelve True/False) a cada elemento.

### ❌ MAL: Loop con Condición

In [ ]:
# Filter even numbers
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
evens = []
for num in numbers:
    if num % 2 == 0:
        evens.append(num)
print(evens)

### ✅ BIEN: Usando filter()

In [ ]:
# Using filter with lambda
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
evens = list(filter(lambda x: x % 2 == 0, numbers))
print(f"Even numbers: {evens}")

# Using filter with named function
def is_even(x: int) -> bool:
    """Check if a number is even."""
    return x % 2 == 0

evens = list(filter(is_even, numbers))
print(f"Even numbers: {evens}")

# Filter with None removes falsy values
mixed_data = [0, 1, False, True, "", "hello", None, [], [1, 2]]
truthy_values = list(filter(None, mixed_data))
print(f"Truthy values: {truthy_values}")

### Ejemplo Práctico: Filtrado de Datos

In [ ]:
# Filter users by age
class User:
    def __init__(self, name: str, age: int, active: bool):
        self.name = name
        self.age = age
        self.active = active
    
    def __repr__(self):
        return f"User({self.name}, {self.age}, active={self.active})"

users = [
    User("Alice", 25, True),
    User("Bob", 17, True),
    User("Charlie", 30, False),
    User("David", 22, True),
    User("Eve", 16, True)
]

# Filter adult active users
def is_adult_active(user: User) -> bool:
    """
    Check if user is adult and active.
    
    :param user: User object
    :type user: User
    :return: True if user is 18+ and active
    :rtype: bool
    """
    return user.age >= 18 and user.active

adult_active_users = list(filter(is_adult_active, users))
print(f"Adult active users: {adult_active_users}")

# Filter valid email addresses
emails = ["user@example.com", "invalid", "test@test.com", "", "admin@site.org"]
valid_emails = list(filter(lambda e: "@" in e and "." in e, emails))
print(f"Valid emails: {valid_emails}")

### Aprendizaje Clave

`filter(function, iterable)` devuelve un iterador con los elementos para los cuales la función devuelve True. Si se pasa None como función, filtra valores falsy. Para casos simples, las list comprehensions con `if` suelen ser más legibles.

**Referencia oficial:** [Python filter() built-in function](https://docs.python.org/3/library/functions.html#filter)

## 3. reduce() - Reducir Colecciones a un Valor

### El Problema que Resuelve

A veces necesitamos combinar todos los elementos de una colección en un solo valor (suma, producto, concatenación, etc.). `reduce()` aplica una función de forma acumulativa a los elementos de un iterable, reduciéndolo a un único valor.

Nota: `reduce()` está en el módulo `functools`, no es una función built-in como `map()` y `filter()`.

### ❌ MAL: Loop con Acumulador

In [ ]:
# Calculate product of all numbers
numbers = [1, 2, 3, 4, 5]
product = 1
for num in numbers:
    product *= num
print(f"Product: {product}")

### ✅ BIEN: Usando reduce()

In [ ]:
from functools import reduce

# Calculate product using reduce
numbers = [1, 2, 3, 4, 5]
product = reduce(lambda x, y: x * y, numbers)
print(f"Product: {product}")

# With initial value
product = reduce(lambda x, y: x * y, numbers, 1)
print(f"Product with initial: {product}")

# Sum (though sum() is better for this)
total = reduce(lambda x, y: x + y, numbers)
print(f"Sum: {total}")

# Find maximum
maximum = reduce(lambda x, y: x if x > y else y, numbers)
print(f"Maximum: {maximum}")

### Ejemplo Práctico: Operaciones Complejas

In [ ]:
from functools import reduce

# Flatten nested lists
nested_lists = [[1, 2], [3, 4], [5, 6]]
flattened = reduce(lambda acc, lst: acc + lst, nested_lists, [])
print(f"Flattened: {flattened}")

# Merge dictionaries
dicts = [{"a": 1}, {"b": 2}, {"c": 3}]
merged = reduce(lambda acc, d: {**acc, **d}, dicts, {})
print(f"Merged dict: {merged}")

# Calculate factorial
def factorial(n: int) -> int:
    """
    Calculate factorial using reduce.
    
    :param n: Number to calculate factorial for
    :type n: int
    :return: Factorial of n
    :rtype: int
    """
    if n == 0:
        return 1
    return reduce(lambda x, y: x * y, range(1, n + 1))

print(f"5! = {factorial(5)}")
print(f"10! = {factorial(10)}")

# Compose functions
def compose(*functions):
    """
    Compose multiple functions into one.
    
    :param functions: Functions to compose
    :return: Composed function
    :rtype: callable
    """
    return reduce(lambda f, g: lambda x: f(g(x)), functions)

# Create a pipeline: add 1, multiply by 2, subtract 3
add_one = lambda x: x + 1
multiply_two = lambda x: x * 2
subtract_three = lambda x: x - 3

pipeline = compose(subtract_three, multiply_two, add_one)
result = pipeline(5)  # ((5 + 1) * 2) - 3 = 9
print(f"Pipeline result: {result}")

### Aprendizaje Clave

`reduce(function, iterable, initial)` aplica una función de dos argumentos acumulativamente a los elementos del iterable, de izquierda a derecha, para reducirlo a un único valor. El parámetro `initial` es opcional pero recomendado para evitar errores con iterables vacíos.

**Referencia oficial:** [functools.reduce()](https://docs.python.org/3/library/functools.html#functools.reduce)

## 4. functools.partial() - Aplicación Parcial de Funciones

### El Problema que Resuelve

A veces tenemos una función con múltiples parámetros, pero queremos crear una versión especializada con algunos parámetros ya fijados. Sin `partial()`, tendríamos que crear funciones wrapper manualmente o usar lambdas, lo que puede ser verboso.

`partial()` permite "congelar" algunos argumentos de una función, creando una nueva función con menos parámetros.

### ❌ MAL: Funciones Wrapper Manuales

In [ ]:
def power(base: float, exponent: float) -> float:
    """Calculate base raised to exponent."""
    return base ** exponent

# Creating specialized functions manually
def square(x: float) -> float:
    """Calculate square of x."""
    return power(x, 2)

def cube(x: float) -> float:
    """Calculate cube of x."""
    return power(x, 3)

print(square(5))
print(cube(3))

### ✅ BIEN: Usando partial()

In [ ]:
from functools import partial

def power(base: float, exponent: float) -> float:
    """Calculate base raised to exponent."""
    return base ** exponent

# Create specialized functions using partial
square = partial(power, exponent=2)
cube = partial(power, exponent=3)

print(f"5 squared: {square(5)}")
print(f"3 cubed: {cube(3)}")

# Partial with positional arguments
def multiply(x: float, y: float, z: float) -> float:
    """Multiply three numbers."""
    return x * y * z

double = partial(multiply, 2)  # Fix first argument to 2
print(f"Double of 5 * 3: {double(5, 3)}")  # 2 * 5 * 3 = 30

### Ejemplo Práctico: Configuración de Funciones

In [ ]:
from functools import partial
import json

# Create specialized JSON dumpers
compact_json = partial(json.dumps, separators=(',', ':'))
pretty_json = partial(json.dumps, indent=4, sort_keys=True)

data = {"name": "Alice", "age": 30, "city": "NYC"}
print("Compact:")
print(compact_json(data))
print("\nPretty:")
print(pretty_json(data))

# Create specialized logging functions
def log_message(message: str, level: str = "INFO", prefix: str = "") -> None:
    """
    Log a message with level and prefix.
    
    :param message: Message to log
    :type message: str
    :param level: Log level
    :type level: str
    :param prefix: Prefix for the message
    :type prefix: str
    """
    print(f"[{level}] {prefix}{message}")

error_log = partial(log_message, level="ERROR", prefix="❌ ")
warning_log = partial(log_message, level="WARNING", prefix="⚠️  ")
info_log = partial(log_message, level="INFO", prefix="ℹ️  ")

error_log("Database connection failed")
warning_log("Deprecated function used")
info_log("Application started successfully")

# Partial with map for data transformation
def convert_temperature(temp: float, from_unit: str, to_unit: str) -> float:
    """
    Convert temperature between units.
    
    :param temp: Temperature value
    :type temp: float
    :param from_unit: Source unit (C or F)
    :type from_unit: str
    :param to_unit: Target unit (C or F)
    :type to_unit: str
    :return: Converted temperature
    :rtype: float
    """
    if from_unit == "C" and to_unit == "F":
        return (temp * 9/5) + 32
    elif from_unit == "F" and to_unit == "C":
        return (temp - 32) * 5/9
    return temp

celsius_to_fahrenheit = partial(convert_temperature, from_unit="C", to_unit="F")
celsius_temps = [0, 10, 20, 30, 40]
fahrenheit_temps = list(map(celsius_to_fahrenheit, celsius_temps))
print(f"\nCelsius: {celsius_temps}")
print(f"Fahrenheit: {fahrenheit_temps}")

### Aprendizaje Clave

`partial(func, *args, **kwargs)` crea una nueva función con algunos argumentos ya fijados. Es útil para crear funciones especializadas, configurar callbacks, y trabajar con APIs que esperan funciones con firmas específicas. Mejora la reutilización de código sin crear wrappers manuales.

**Referencia oficial:** [functools.partial()](https://docs.python.org/3/library/functools.html#functools.partial)

## 5. functools.lru_cache() - Memoización para Optimización

### El Problema que Resuelve

Muchas funciones realizan cálculos costosos que se repiten con los mismos argumentos. Sin caché, estos cálculos se ejecutan cada vez, desperdiciando tiempo y recursos. La memoización (caching de resultados) puede mejorar dramáticamente el rendimiento.

`lru_cache()` es un decorador que implementa un caché LRU (Least Recently Used) automáticamente, guardando los resultados de llamadas anteriores.

### ❌ MAL: Sin Caché (Ineficiente)

In [ ]:
import time

def fibonacci_slow(n: int) -> int:
    """Calculate Fibonacci number (slow recursive version)."""
    if n < 2:
        return n
    return fibonacci_slow(n - 1) + fibonacci_slow(n - 2)

# This will be very slow for n > 30
start = time.time()
result = fibonacci_slow(35)
end = time.time()
print(f"fibonacci_slow(35) = {result}")
print(f"Time: {end - start:.4f}s")

### ✅ BIEN: Con lru_cache()

In [ ]:
from functools import lru_cache
import time

@lru_cache(maxsize=None)  # Unlimited cache size
def fibonacci_fast(n: int) -> int:
    """
    Calculate Fibonacci number with memoization.
    
    :param n: Position in Fibonacci sequence
    :type n: int
    :return: Fibonacci number at position n
    :rtype: int
    """
    if n < 2:
        return n
    return fibonacci_fast(n - 1) + fibonacci_fast(n - 2)

# This will be extremely fast even for large n
start = time.time()
result = fibonacci_fast(35)
end = time.time()
print(f"fibonacci_fast(35) = {result}")
print(f"Time: {end - start:.6f}s")

# Check cache statistics
print(f"\nCache info: {fibonacci_fast.cache_info()}")

# Can even calculate much larger values
start = time.time()
result = fibonacci_fast(100)
end = time.time()
print(f"\nfibonacci_fast(100) = {result}")
print(f"Time: {end - start:.6f}s")
print(f"Cache info: {fibonacci_fast.cache_info()}")

### Configuración de lru_cache

In [ ]:
from functools import lru_cache

# Limited cache size (LRU eviction when full)
@lru_cache(maxsize=128)
def expensive_computation(x: int, y: int) -> int:
    """
    Simulate expensive computation.
    
    :param x: First parameter
    :type x: int
    :param y: Second parameter
    :type y: int
    :return: Result of computation
    :rtype: int
    """
    print(f"Computing {x} + {y}...")
    return x + y

# First calls compute
print(expensive_computation(1, 2))
print(expensive_computation(3, 4))

# Repeated calls use cache
print(expensive_computation(1, 2))  # No "Computing" message
print(expensive_computation(3, 4))  # No "Computing" message

# Clear cache manually
expensive_computation.cache_clear()
print("\nCache cleared!")
print(expensive_computation(1, 2))  # Computes again

### Ejemplo Práctico: Optimización de Consultas

In [ ]:
from functools import lru_cache
import time

# Simulate database query
@lru_cache(maxsize=100)
def get_user_data(user_id: int) -> dict:
    """
    Fetch user data (simulated with cache).
    
    :param user_id: User ID to fetch
    :type user_id: int
    :return: User data dictionary
    :rtype: dict
    """
    print(f"Fetching user {user_id} from database...")
    time.sleep(0.1)  # Simulate network delay
    return {"id": user_id, "name": f"User{user_id}", "email": f"user{user_id}@example.com"}

# First access - slow
start = time.time()
user = get_user_data(123)
print(f"User: {user}")
print(f"Time: {time.time() - start:.3f}s\n")

# Second access - instant (cached)
start = time.time()
user = get_user_data(123)
print(f"User: {user}")
print(f"Time: {time.time() - start:.6f}s\n")

# Calculate prime numbers with cache
@lru_cache(maxsize=None)
def is_prime(n: int) -> bool:
    """
    Check if number is prime.
    
    :param n: Number to check
    :type n: int
    :return: True if prime
    :rtype: bool
    """
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

# Find primes up to 100
primes = [n for n in range(100) if is_prime(n)]
print(f"Primes up to 100: {primes}")
print(f"Cache info: {is_prime.cache_info()}")

### Aprendizaje Clave

`@lru_cache(maxsize=128)` cachea los resultados de una función basándose en sus argumentos. `maxsize=None` permite caché ilimitado. Es ideal para funciones puras (sin efectos secundarios) con cálculos costosos. Usa `cache_info()` para estadísticas y `cache_clear()` para limpiar el caché.

**Referencia oficial:** [functools.lru_cache()](https://docs.python.org/3/library/functools.html#functools.lru_cache)

## 6. Combinando Técnicas Funcionales

### Pipeline de Procesamiento de Datos

Las técnicas funcionales brillan cuando se combinan para crear pipelines de transformación de datos elegantes y eficientes.

In [ ]:
from functools import reduce, partial, lru_cache

# Sample data: list of transactions
transactions = [
    {"id": 1, "amount": 100, "type": "credit", "category": "salary"},
    {"id": 2, "amount": 50, "type": "debit", "category": "food"},
    {"id": 3, "amount": 200, "type": "credit", "category": "bonus"},
    {"id": 4, "amount": 30, "type": "debit", "category": "transport"},
    {"id": 5, "amount": 150, "type": "credit", "category": "salary"},
    {"id": 6, "amount": 80, "type": "debit", "category": "food"},
]

# Pipeline: filter credits -> map to amounts -> reduce to sum
is_credit = lambda t: t["type"] == "credit"
get_amount = lambda t: t["amount"]
add = lambda x, y: x + y

total_credits = reduce(
    add,
    map(get_amount, filter(is_credit, transactions)),
    0
)
print(f"Total credits: ${total_credits}")

# More complex pipeline with partial
def filter_by_type(transaction: dict, trans_type: str) -> bool:
    """Filter transactions by type."""
    return transaction["type"] == trans_type

def filter_by_category(transaction: dict, category: str) -> bool:
    """Filter transactions by category."""
    return transaction["category"] == category

# Create specialized filters
is_debit = partial(filter_by_type, trans_type="debit")
is_food = partial(filter_by_category, category="food")

# Calculate total food expenses
food_expenses = reduce(
    add,
    map(get_amount, filter(lambda t: is_debit(t) and is_food(t), transactions)),
    0
)
print(f"Total food expenses: ${food_expenses}")

# Group by category using reduce
def group_by_category(acc: dict, transaction: dict) -> dict:
    """
    Group transactions by category.
    
    :param acc: Accumulator dictionary
    :type acc: dict
    :param transaction: Transaction to add
    :type transaction: dict
    :return: Updated accumulator
    :rtype: dict
    """
    category = transaction["category"]
    if category not in acc:
        acc[category] = []
    acc[category].append(transaction)
    return acc

grouped = reduce(group_by_category, transactions, {})
print(f"\nGrouped by category:")
for category, trans in grouped.items():
    total = sum(t["amount"] for t in trans)
    print(f"  {category}: {len(trans)} transactions, total ${total}")

### Aprendizaje Clave

Combinar `map()`, `filter()`, `reduce()`, `partial()` y `lru_cache()` permite crear pipelines de procesamiento de datos expresivos y eficientes. Cada función tiene un propósito específico, y juntas forman un toolkit poderoso para programación funcional en Python.

**Referencia oficial:** [Functional Programming HOWTO](https://docs.python.org/3/howto/functional.html)

## Ejercicios Prácticos

### Ejercicio 1: Pipeline de Transformación (Básico)

Dada una lista de números, crea un pipeline que:
1. Filtre solo los números pares
2. Eleve cada número al cuadrado
3. Sume todos los resultados

Usa `filter()`, `map()` y `reduce()`.

In [ ]:
from functools import reduce

# Tu código aquí
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Implementa el pipeline
result = 0  # Reemplaza con tu solución

print(f"Result: {result}")  # Should be 220 (2² + 4² + 6² + 8² + 10²)

### Ejercicio 2: Funciones Especializadas con partial() (Intermedio)

Crea una función `calculate_price()` que tome:
- `base_price`: precio base
- `tax_rate`: tasa de impuesto (decimal)
- `discount`: descuento (decimal)

Luego usa `partial()` para crear:
- `calculate_price_with_vat`: con IVA del 21%
- `calculate_price_with_discount`: con 10% de descuento
- `calculate_final_price`: con IVA del 21% y 10% de descuento

In [ ]:
from functools import partial

# Tu código aquí
def calculate_price(base_price, tax_rate=0, discount=0):
    pass

# Crea las funciones especializadas
calculate_price_with_vat = None
calculate_price_with_discount = None
calculate_final_price = None

# Prueba tu código
print(f"Base price: $100")
print(f"With VAT: ${calculate_price_with_vat(100)}")
print(f"With discount: ${calculate_price_with_discount(100)}")
print(f"Final price: ${calculate_final_price(100)}")

### Ejercicio 3: Optimización con lru_cache() (Avanzado)

Implementa una función `count_paths()` que calcule el número de caminos únicos en una cuadrícula de m×n desde la esquina superior izquierda hasta la esquina inferior derecha (solo puedes moverte hacia abajo o hacia la derecha).

1. Primero implementa la versión recursiva sin caché
2. Luego agrega `@lru_cache` y compara el rendimiento
3. Calcula los caminos para una cuadrícula de 20×20

In [ ]:
from functools import lru_cache
import time

# Tu código aquí
def count_paths_slow(m, n):
    """Count paths without cache."""
    pass

@lru_cache(maxsize=None)
def count_paths_fast(m, n):
    """Count paths with cache."""
    pass

# Prueba y compara
print("Testing 10x10 grid:")
start = time.time()
result_slow = count_paths_slow(10, 10)
time_slow = time.time() - start
print(f"Without cache: {result_slow} paths in {time_slow:.4f}s")

start = time.time()
result_fast = count_paths_fast(10, 10)
time_fast = time.time() - start
print(f"With cache: {result_fast} paths in {time_fast:.6f}s")
print(f"Speedup: {time_slow/time_fast:.0f}x faster")

# Now try 20x20 (only with cache!)
print(f"\n20x20 grid: {count_paths_fast(20, 20)} paths")

## Resumen

En este notebook hemos aprendido:

1. **map()**: Aplica una función a cada elemento de un iterable, ideal para transformaciones uniformes de datos

2. **filter()**: Selecciona elementos que cumplen una condición, proporcionando una forma funcional de filtrado

3. **reduce()**: Combina todos los elementos de un iterable en un único valor mediante una función acumulativa

4. **functools.partial()**: Crea funciones especializadas fijando algunos argumentos, mejorando la reutilización de código

5. **functools.lru_cache()**: Implementa memoización automática para optimizar funciones con cálculos costosos

La programación funcional en Python permite escribir código más expresivo, componible y eficiente. Aunque Python no es un lenguaje puramente funcional, estas herramientas son invaluables para procesamiento de datos, optimización de rendimiento, y creación de pipelines de transformación elegantes.

## Preguntas de Autoevaluación

### 1. ¿Cuál es la diferencia principal entre map() y una list comprehension?

**Respuesta:** `map()` devuelve un iterador lazy (no evalúa hasta que se consume), mientras que una list comprehension crea una lista inmediatamente. `map()` es más eficiente en memoria para grandes datasets, pero las list comprehensions suelen ser más legibles y Pythonic para casos simples.

### 2. ¿Por qué reduce() está en functools y no es una función built-in como map() y filter()?

**Respuesta:** Guido van Rossum consideró que `reduce()` es menos legible que alternativas como loops o funciones específicas (`sum()`, `any()`, `all()`). Se movió a `functools` en Python 3 para desalentar su uso excesivo, aunque sigue siendo útil para operaciones de reducción complejas.

### 3. ¿Cuándo usarías partial() en lugar de una lambda?

**Respuesta:** Usa `partial()` cuando necesites: (1) crear funciones reutilizables con nombre, (2) preservar la firma de la función original, (3) trabajar con funciones que esperan callables específicos, o (4) cuando la lambda sería demasiado compleja. `partial()` es más explícito y mantenible.

### 4. ¿Qué significa "LRU" en lru_cache y por qué es importante?

**Respuesta:** LRU significa "Least Recently Used" (Menos Recientemente Usado). Cuando el caché alcanza su tamaño máximo, elimina el elemento que no se ha usado por más tiempo. Esto es importante para controlar el uso de memoria mientras se mantiene en caché los datos más relevantes.

### 5. ¿Qué tipo de funciones son buenas candidatas para lru_cache()?

**Respuesta:** Funciones puras (sin efectos secundarios) con: (1) cálculos costosos, (2) argumentos hashables, (3) llamadas repetidas con los mismos argumentos, y (4) número limitado de combinaciones de argumentos. No uses caché en funciones con efectos secundarios o que dependen de estado externo.

### 6. ¿Cómo combinarías map(), filter() y reduce() para procesar una lista de diccionarios?

**Respuesta:** Primero usa `filter()` para seleccionar diccionarios que cumplan condiciones, luego `map()` para extraer o transformar valores específicos, y finalmente `reduce()` para combinar los resultados en un valor único. Ejemplo: filtrar usuarios activos, mapear a edades, reducir a edad promedio.

### 7. ¿Cuál es la ventaja de usar funciones de orden superior sobre loops tradicionales?

**Respuesta:** Las funciones de orden superior: (1) son más declarativas (expresan QUÉ hacer, no CÓMO), (2) son más componibles (fáciles de combinar), (3) reducen errores (menos estado mutable), (4) son más fáciles de paralelizar, y (5) comunican intención más claramente. Sin embargo, los loops pueden ser más legibles para lógica compleja.

## Recursos y Referencias Oficiales

### Documentación Oficial
- **[Python map() built-in](https://docs.python.org/3/library/functions.html#map)**: https://docs.python.org/3/library/functions.html#map
  - Documentación completa de la función map()

- **[Python filter() built-in](https://docs.python.org/3/library/functions.html#filter)**: https://docs.python.org/3/library/functions.html#filter
  - Documentación completa de la función filter()

- **[functools module](https://docs.python.org/3/library/functools.html)**: https://docs.python.org/3/library/functools.html
  - Módulo completo con reduce(), partial(), lru_cache() y más

- **[functools.reduce()](https://docs.python.org/3/library/functools.html#functools.reduce)**: https://docs.python.org/3/library/functools.html#functools.reduce
  - Documentación específica de reduce()

- **[functools.partial()](https://docs.python.org/3/library/functools.html#functools.partial)**: https://docs.python.org/3/library/functools.html#functools.partial
  - Documentación de aplicación parcial de funciones

- **[functools.lru_cache()](https://docs.python.org/3/library/functools.html#functools.lru_cache)**: https://docs.python.org/3/library/functools.html#functools.lru_cache
  - Documentación del decorador de caché LRU

### Guías y Tutoriales
- **[Functional Programming HOWTO](https://docs.python.org/3/howto/functional.html)**: https://docs.python.org/3/howto/functional.html
  - Guía oficial sobre programación funcional en Python

- **[Python Functional Programming](https://realpython.com/python-functional-programming/)**: https://realpython.com/python-functional-programming/
  - Tutorial completo de Real Python sobre programación funcional

### Mejores Prácticas
- **[PEP 289 - Generator Expressions](https://www.python.org/dev/peps/pep-0289)**: https://www.python.org/dev/peps/pep-0289
  - PEP sobre expresiones generadoras, alternativa a map() y filter()

### Herramientas Relacionadas
- **[itertools module](https://docs.python.org/3/library/itertools.html)**: https://docs.python.org/3/library/itertools.html
  - Módulo con herramientas adicionales para iteradores funcionales

- **[operator module](https://docs.python.org/3/library/operator.html)**: https://docs.python.org/3/library/operator.html
  - Funciones que corresponden a operadores built-in, útiles con map/filter/reduce

### Notas Importantes
- Todos los enlaces están actualizados a partir de 2025
- Se recomienda revisar la documentación oficial regularmente
- Para datasets grandes, considera usar bibliotecas especializadas como pandas o NumPy
- La programación funcional en Python es un complemento, no un reemplazo de otros paradigmas